# Flan-T5 Prompts

In [20]:
!nvidia-smi

Thu Feb  9 17:12:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A4000    Off  | 00000000:00:05.0 Off |                  Off |
| 41%   33C    P8    19W / 140W |  12387MiB / 16376MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:           44Gi       4.3Gi        18Gi        38Mi        21Gi        39Gi
Swap:            0B          0B          0B


In [4]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          8
On-line CPU(s) list:             0-7
Thread(s) per core:              1
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           106
Model name:                      Intel(R) Xeon(R) Gold 5315Y CPU @ 3.20GHz
Stepping:                        6
CPU MHz:                         3199.910
BogoMIPS:                        6400.17
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       384 KiB
L1i cache:                       256 KiB
L2 cache:                        10 MiB
L3 cache:                        96 MiB
NUMA node0 CPU(s):               0-7
V

In [5]:
! pip install -q transformers accelerate sentencepiece gradio

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-xl")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl").to("cuda")

In [4]:
def generate_simple(input_text):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
  output = model.generate(
    input_ids, 
    max_length=1024,
    temperature=0.7)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [25]:
input_text = """Explain artificial intelligence"""
generate_simple(input_text)

'artificial intelligence'

In [26]:
def generate_custom(input_text):
  input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
  output = model.generate(
    input_ids, 
    min_length=256,
    max_new_tokens=1024,
    length_penalty=1.4,
    no_repeat_ngram_size=2,
    top_k=150,
    top_p=0.92,
    repetition_penalty=2.1,
    #num_beams=4,
    temperature=0.7)
  return tokenizer.decode(output[0], skip_special_tokens=True)

In [27]:
generate_custom(input_text)

'artificial intelligence is a branch of computer science that deals with the development of machines that can learn and perform tasks. It is also known as machine learning. Artificial intelligence has been around for centuries, but it was only in the last few decades that people began to realize that it could be used to create intelligent systems. Today, AI encompasses everything from robots to computers. In this article, we will look at some of the most important developments in AI. The first major breakthrough in artificially intelligent machines was made in 1956 by John McCarthy. He developed an algorithm that allowed computers to recognize faces and objects. This was called face recognition. Another key advance was the creation of deep neural networks. These were designed to allow computers (and other devices) to learn from each other. Deep neural nets are now widely used in many areas of computing. They have become so sophisticated that they can even read human language. A recent 

In [ ]:
input_text = """
Answer the following question by reasoning step by step.
The cafeteria had 23 apples. If the used 20 for lunch and bougth 6 more, how many apples do they have?
"""
generate(input_text)

In [30]:

input_text = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.

QUESTION: Which state/country's law governs the interpretation of the contract?
=========
Content: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.
Source: 28-pl
Content: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a waiver of such (or any other)  right or remedy.\n\n11.7 Severability. The invalidity, illegality or unenforceability of any term (or part of a term) of this Agreement shall not affect the continuation  in force of the remainder of the term (if any) and this Agreement.\n\n11.8 No Agency. Except as expressly stated otherwise, nothing in this Agreement shall create an agency, partnership or joint venture of any  kind between the parties.\n\n11.9 No Third-Party Beneficiaries.
Source: 30-pl
Content: (b) if Google believes, in good faith, that the Distributor has violated or caused Google to violate any Anti-Bribery Laws (as  defined in Clause 8.5) or that such a violation is reasonably likely to occur,
Source: 4-pl
=========
FINAL ANSWER:"""
generate(input_text)

'English law'

In [31]:
input_text = """
Q: Answer the following yes/no question by
reasoning step-by-step.
Could a dandelion suffer from hepatitis?
A: Hepatitis only affects organisms with livers.
Dandelions don’t have a liver. The answer is no.
Q: Answer the following yes/no question by
reasoning step-by-step.
Can a monkey write a post?
A: 
"""

generate(input_text)

"Monkeys can't write. Monkeys can't read. Monkeys can't write. The answer is no."

In [40]:
input_text = """
Generate a question for the following text: The Python plugin allows you to run python scripts on the o9 Platform. It acts like a bridge between the GraphCube server and the Hadoop ecosystem on which the python scripts are executed. One of the preferred languages of the data scientists is python. The data size for AI/ML is huge and is in the order of millions and billions rows of data. The o9 Platform provides an easy way to provide the metadata configuration option on the Platform UI and provides scripts for configuration to run on the Hadoop cluster at scale."""
generate(input_text)

'What is the name of the language that is used by data scientists?'